In [96]:
import pandas as pd
import numpy as np
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
from scipy import stats
from sklearn.model_selection import train_test_split

In [2]:
# con = psycopg2.connect(
#     host = 'mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com',
#     database = 'mid_term_project',
#     user = 'lhl_student',
#     password = 'lhl_student',
#     port = 5432
# )
# sql_flight_info = "SELECT fl_date, mkt_unique_carrier, op_unique_carrier, origin, dep_time, dep_delay, dest, arr_time, arr_delay, crs_arr_time, actual_elapsed_time, distance, carrier_delay, weather_delay, nas_delay, security_delay, late_aircraft_delay FROM flights WHERE (origin = 'ORD' or origin = 'ATL' or origin = 'DFW' or origin = 'DEN' or origin = 'LAX' or origin = 'CLT' or origin = 'SEA' or origin = 'SFO' or origin = 'EWR' or origin = 'PHX' or origin = 'LGA' or origin = 'IAH' or origin = 'LAS' or origin = 'BOS' or origin = 'PHL' or origin = 'MCO' or origin = 'DTW' or origin = 'MSP' or origin = 'DCA') or (dest = 'ORD' or dest = 'ATL' or dest = 'DFW' or dest = 'DEN' or dest = 'LAX' or dest = 'CLT' or dest = 'SEA' or dest = 'SFO' or dest = 'EWR' or dest = 'PHX' or dest = 'LGA' or dest = 'IAH' or dest = 'LAS' or dest = 'BOS' or dest = 'PHL' or dest = 'MCO' or dest = 'DTW' or dest = 'MSP' or dest = 'DCA') ORDER BY RANDOM() LIMIT 1000000;"
# data_flight_info = pd.read_sql_query(sql_flight_info,con)
# con.close()

In [3]:
# data_flight_info.to_csv('training_data.csv', encoding='utf-8', index=False)

In [114]:
data_flight_info = pd.read_csv('/Users/craiglynch/Documents/GitHub/Predicting-Flight-Delays/data/training_data.csv', sep = ',')

In [115]:
# Creating a function which will remove extra leading 
# and tailing whitespace from the data.
# pass dataframe as a parameter here

def whitespace_remover(dataframe):
    
    # iterating over the columns
    for i in dataframe.columns:
          
        # checking datatype of each columns
        if dataframe[i].dtype == 'object':
              
            # applying strip function on column
            dataframe[i] = dataframe[i].map(str.strip)
        else:
              
            # if condn. is False then it will do nothing.
            pass

In [116]:
df_flight_info = data_flight_info.copy()
whitespace_remover(df_flight_info)
df_flight_info.shape

(1000000, 17)

In [117]:
# Creating year, month, date columns
# Create new columns
df_flight_info['year'] = pd.DatetimeIndex(df_flight_info['fl_date']).year
df_flight_info['month'] = pd.DatetimeIndex(df_flight_info['fl_date']).month
df_flight_info['day'] = pd.DatetimeIndex(df_flight_info['fl_date']).day
df_flight_info['weekday'] = ((pd.DatetimeIndex(df_flight_info['fl_date']).dayofweek) // 5 == 1).astype(float)
df_flight_info.isnull().sum()

fl_date                     0
mkt_unique_carrier          0
op_unique_carrier           0
origin                      0
dep_time                16721
dep_delay               17024
dest                        0
arr_time                17739
arr_delay               20212
crs_arr_time                0
actual_elapsed_time     20025
distance                    0
carrier_delay          805481
weather_delay          805481
nas_delay              805481
security_delay         805481
late_aircraft_delay    805481
year                        0
month                       0
day                         0
weekday                     0
dtype: int64

In [118]:
# Filling all NaN delay info with 0
df_flight_info[['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']] = df_flight_info[['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].fillna(0)

In [119]:
# Because dep_time, dep_delay, arr_time, arr_delay, and actual_elapsed_time have relatively small amounts of NaN they were dropped
df_flight_info = df_flight_info.dropna()

In [120]:
df_flight_info.isnull().sum()

fl_date                0
mkt_unique_carrier     0
op_unique_carrier      0
origin                 0
dep_time               0
dep_delay              0
dest                   0
arr_time               0
arr_delay              0
crs_arr_time           0
actual_elapsed_time    0
distance               0
carrier_delay          0
weather_delay          0
nas_delay              0
security_delay         0
late_aircraft_delay    0
year                   0
month                  0
day                    0
weekday                0
dtype: int64

In [121]:
# 20,494 rows removed
df_flight_info.shape

(979506, 21)

In [122]:
df_flight_info1 = df_flight_info[df_flight_info['carrier_delay'] <= 720]
df_flight_info2 = df_flight_info1[df_flight_info1['weather_delay'] <= 720]
df_flight_info3 = df_flight_info2[df_flight_info2['nas_delay'] <= 720]
df_flight_info4 = df_flight_info3[df_flight_info3['security_delay'] <= 720]
df_flight_info = df_flight_info4[df_flight_info4['late_aircraft_delay'] <= 720]
#630 rows removed

In [123]:
df_flight_info.shape

(978876, 21)

In [127]:
# Putting arr_time and dep_time into datetime format
df_flight_info['dep_time'] = df_flight_info['dep_time'].astype(int)
df_flight_info['dep_time'] = df_flight_info['dep_time'].apply(lambda x: '{0:0>4}'.format(x)).astype(str)
df_flight_info['dep_time'] = df_flight_info['dep_time'].str.replace(r"\\*(\d{2}(?!$))", r"\1:")
df_flight_info['dep_time'] = df_flight_info['dep_time'].replace('24:00', '00:00')
df_flight_info['dep_time'] = pd.to_datetime(df_flight_info['dep_time'],format= '%H:%M' ).dt.time

<ipython-input-127-56dbe100a4dd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_flight_info['dep_time'] = df_flight_info['dep_time'].astype(int)
<ipython-input-127-56dbe100a4dd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_flight_info['dep_time'] = df_flight_info['dep_time'].apply(lambda x: '{0:0>4}'.format(x)).astype(str)
<ipython-input-127-56dbe100a4dd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [128]:
df_flight_info['arr_time'] = df_flight_info['arr_time'].astype(int)
df_flight_info['arr_time'] = df_flight_info['arr_time'].apply(lambda x: '{0:0>4}'.format(x)).astype(str)
df_flight_info['arr_time'] = df_flight_info['arr_time'].str.replace(r"\\*(\d{2}(?!$))", r"\1:")
df_flight_info['arr_time'] = df_flight_info['arr_time'].replace('24:00', '00:00')
df_flight_info['arr_time'] = pd.to_datetime(df_flight_info['arr_time'],format= '%H:%M' ).dt.time

<ipython-input-128-35c43b1bd7bd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_flight_info['arr_time'] = df_flight_info['arr_time'].astype(int)
<ipython-input-128-35c43b1bd7bd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_flight_info['arr_time'] = df_flight_info['arr_time'].apply(lambda x: '{0:0>4}'.format(x)).astype(str)
<ipython-input-128-35c43b1bd7bd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [129]:
df_flight_info['crs_arr_time'] = df_flight_info['crs_arr_time'].astype(int)
df_flight_info['crs_arr_time'] = df_flight_info['crs_arr_time'].apply(lambda x: '{0:0>4}'.format(x)).astype(str)
df_flight_info['crs_arr_time'] = df_flight_info['crs_arr_time'].str.replace(r"\\*(\d{2}(?!$))", r"\1:")
df_flight_info['crs_arr_time'] = df_flight_info['crs_arr_time'].replace('24:00', '00:00')
df_flight_info['crs_arr_time'] = pd.to_datetime(df_flight_info['crs_arr_time'],format= '%H:%M' ).dt.time

<ipython-input-129-02c718e33a92>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_flight_info['crs_arr_time'] = df_flight_info['crs_arr_time'].astype(int)
<ipython-input-129-02c718e33a92>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_flight_info['crs_arr_time'] = df_flight_info['crs_arr_time'].apply(lambda x: '{0:0>4}'.format(x)).astype(str)
<ipython-input-129-02c718e33a92>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [130]:
df_flight_info.head()

,fl_date,mkt_unique_carrier,op_unique_carrier,origin,dep_time,dep_delay,dest,arr_time,arr_delay,crs_arr_time,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,year,month,day,weekday
0,2018-08-26,AS,AS,PDX,17:17:00,-3.0,LAX,19:36:00,-5.0,19:41:00,...,834.0,0.0,0.0,0.0,0.0,0.0,2018,8,26,1.0
1,2018-05-24,UA,OO,DEN,17:50:00,0.0,PSP,19:13:00,3.0,19:10:00,...,776.0,0.0,0.0,0.0,0.0,0.0,2018,5,24,0.0
2,2019-01-10,AA,PT,CLT,11:05:00,-4.0,ROA,12:00:00,-16.0,12:16:00,...,155.0,0.0,0.0,0.0,0.0,0.0,2019,1,10,0.0
3,2018-07-09,WN,WN,ELP,05:54:00,-1.0,LAS,06:23:00,-12.0,06:35:00,...,583.0,0.0,0.0,0.0,0.0,0.0,2018,7,9,0.0
4,2019-07-09,DL,9E,LGA,16:29:00,0.0,BNA,17:59:00,-12.0,18:11:00,...,764.0,0.0,0.0,0.0,0.0,0.0,2019,7,9,0.0


In [131]:
data_flight_info.to_csv('training_data_outliers_removed.csv', encoding='utf-8', index=False)

In [ ]:
y = data_flight_info['arr_delay']
X = data_flight_info.drop(['arr_delay'],axis=1)